# jupyter lab --ip=0.0.0.0
# start-master.sh
# start-worker.sh spark://yash-kukrejade-6:7077

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# from F import col, explode, lower, split

spark_session = SparkSession.builder\
        .master("spark://waldo-ruiz-master-de-6:7077") \
        .appName("Lecture1_Example0_with_spark")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores", 2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/13 10:00:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
lines = spark_context.textFile('/home/ubuntu/i-have-a-dream.txt')
lines.first()

24/03/13 10:06:07 INFO MemoryStore: Block broadcast_8 stored as values in memory (estimated size 221.5 KiB, free 433.2 MiB)
24/03/13 10:06:07 INFO MemoryStore: Block broadcast_8_piece0 stored as bytes in memory (estimated size 32.6 KiB, free 433.1 MiB)
24/03/13 10:06:07 INFO BlockManagerInfo: Added broadcast_8_piece0 in memory on waldo-ruiz-master-de-6:10005 (size: 32.6 KiB, free: 434.2 MiB)
24/03/13 10:06:07 INFO SparkContext: Created broadcast 8 from textFile at NativeMethodAccessorImpl.java:0
24/03/13 10:06:07 INFO FileInputFormat: Total input files to process : 1
24/03/13 10:06:07 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/03/13 10:06:07 INFO DAGScheduler: Got job 4 (runJob at PythonRDD.scala:181) with 1 output partitions
24/03/13 10:06:07 INFO DAGScheduler: Final stage: ResultStage 4 (runJob at PythonRDD.scala:181)
24/03/13 10:06:07 INFO DAGScheduler: Parents of final stage: List()
24/03/13 10:06:07 INFO DAGScheduler: Missing parents: List()
24/03/13 10:06:0

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 4 times, most recent failure: Lost task 0.3 in stage 4.0 (TID 19) (192.168.2.113 executor 2): java.io.InvalidClassException: org.apache.spark.rdd.RDD; local class incompatible: stream classdesc serialVersionUID = 823754013007382808, local class serialVersionUID = 3516924559342767982
	at java.base/java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:560)
	at java.base/java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:2020)
	at java.base/java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1870)
	at java.base/java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:2020)
	at java.base/java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1870)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2201)
	at java.base/java

24/03/13 10:06:38 INFO StandaloneSchedulerBackend: Requesting to kill executor(s) 2
24/03/13 10:06:38 INFO StandaloneSchedulerBackend: Actual list of executor(s) to be killed is 2
24/03/13 10:06:38 INFO ExecutorAllocationManager: Executors 2 removed due to idle timeout.
24/03/13 10:06:42 INFO SparkContext: Invoking stop() from shutdown hook
24/03/13 10:06:42 INFO SparkContext: SparkContext is stopping with exitCode 0.
24/03/13 10:06:42 INFO SparkUI: Stopped Spark web UI at http://waldo-ruiz-master-de-6:4040
24/03/13 10:06:42 INFO StandaloneSchedulerBackend: Shutting down all executors
24/03/13 10:06:42 INFO StandaloneSchedulerBackend$StandaloneDriverEndpoint: Asking each executor to shut down
24/03/13 10:06:42 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
24/03/13 10:06:43 INFO MemoryStore: MemoryStore cleared
24/03/13 10:06:43 INFO BlockManager: BlockManager stopped
24/03/13 10:06:43 INFO BlockManagerMaster: BlockManagerMaster stopped
24/03/13 10:06:43 I

In [29]:
# root
#  |-- author: string (nullable = true) 
#  |-- body: string (nullable = true)
#  |-- content: string (nullable = true)
#  |-- content_len: long (nullable = true)
#  |-- id: string (nullable = true)
#  |-- normalizedBody: string (nullable = true)
#  |-- subreddit: string (nullable = true)
#  |-- subreddit_id: string (nullable = true)
#  |-- summary: string (nullable = true)
#  |-- summary_len: long (nullable = true) 
#  |-- title: string (nullable = true)

reddit_data = spark_session.read.json("/home/ubuntu/input/sample-2000.json")
reddit_data = reddit_data.drop(
    *[
        "content_len",
        "summary_len",
        "id",
        "subreddit_id",
        "body",
        "content",
        "summary",
        "title",
        "subreddit",
        "author"
    ]
)
reddit_data.show()

+------------------+--------------------+--------------------+-----------+-------+--------------------+--------------------+------------+--------------------+-----------+--------+
|            author|                body|             content|content_len|     id|      normalizedBody|           subreddit|subreddit_id|             summary|summary_len|   title|
+------------------+--------------------+--------------------+-----------+-------+--------------------+--------------------+------------+--------------------+-----------+--------+
|  raysofdarkmatter|I think it should...|I think it should...|        178|c69al3r|I think it should...|                math|    t5_2qh0n|Shifting seasonal...|          8|    null|
|           Stork13|Art is about the ...|Art is about the ...|        148|c6a9nxd|Art is about the ...|               funny|    t5_2qh33|Personal opinions...|          4|    null|
|     Cloud_dreamer|Ask me what I thi...|Ask me what I thi...|         76|c6acx4l|Ask me what I thi.

In [30]:
# https://www.cs.cmu.edu/~biglou/resources/bad-words.txt
bad_words = spark_context.textFile("/home/ubuntu/bad_words.txt")
# bad_words = spark_session.createDataFrame(bad_words, ['bad_words'])
bad_words = bad_words.filter(lambda x: x != '').collect()
# print(bad_words)

In [31]:
# reddit_data = reddit_data.withColumn("splited_words", F.split("normalizedBody", ' '))
reddit_data = reddit_data.filter(F.col('normalizedBody').rlike('|'.join(bad_words)))

In [32]:
# Read bad words from text file
bad_words = spark_session.read.text("/home/ubuntu/bad_words.txt") \
    .filter(F.col("value") != "") \
    .select(F.col("value").alias("bad_word"))

# Extract words from comments and explode them
words = reddit_data.select(F.explode(F.split(F.lower(F.col("normalizedBody")), "\\s+")).alias("word"))

# Filter out bad words
bad_words_counts = words.join(bad_words, words.word == bad_words.bad_word, "left_outer") \
    .filter(F.col("bad_word").isNotNull()) \
    .groupBy("bad_word") \
    .count() \
    .orderBy("count", ascending=False)

# # Show bad words counts
bad_words_counts.show()

# Stop SparkSession
spark_session.stop()


+--------+-----+
|bad_word|count|
+--------+-----+
|    shit|  176|
| fucking|  170|
|    fuck|  127|
|     god|  103|
|    kill|   90|
|     kid|   87|
|   black|   86|
|     sex|   80|
|  stupid|   71|
|     ass|   66|
|    damn|   63|
|   fight|   59|
|   girls|   56|
|     fat|   54|
|   death|   52|
|     gun|   50|
|    hell|   49|
|     gay|   49|
|  killed|   46|
|  fucked|   45|
+--------+-----+
only showing top 20 rows

